In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from file_utils import load_dataframes

In [2]:
alley_inventory, categories, items, transaction_items__prior, transaction_items__train, transactions = load_dataframes('data')

In [3]:
alley_inventory.info()
alley_inventory.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 134 entries, 0 to 133
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   alley_id  134 non-null    int64 
 1   alley     134 non-null    object
dtypes: int64(1), object(1)
memory usage: 2.2+ KB


,alley_id,alley
0,1,prepared soups salads
1,2,specialty cheeses
2,3,energy granola bars
3,4,instant foods
4,5,marinades meat preparation
